In [ ]:
from utils import *
# BEST_MODEL_DIR = BEST_MODEL_DIR
# BEST_MODEL_DIR = 'data/2637759/'
# BEST_MODEL_DIR = 'data/2637760/'
# BEST_MODEL_DIR = 'data/2637771/'
# BEST_MODEL_DIR = 'data/2637772/'

net_path = f'{BEST_MODEL_DIR}/best_l3.pth' # path to the model

N_PLOTS = 1 # number of plots to generate, actually a multiplier

ds = LiuqeDataset(EVAL_DS_PATH)

m = LiuqeNet(InputNet(ds.x_mean_std), GridNet(), FluxHead(), FluxHead(), LCFSHead()) # initialize model
m.load_state_dict(torch.load(net_path, map_location=torch.device("cpu"))) # load pretrained model

lcfs = LCFSNet(m.input_net, m.lcfs_head) # initialize LCFS model

for i in range(N_PLOTS): plot_network_outputs(ds, m, title=f'TEST {i}') # plot network outputs

for i in range(N_PLOTS): plot_lcfs_net_out(ds, lcfs, title=f'TEST {i}') # plot LCFS outputs

In [ ]:
# testing batched input tests
import torch
import torch.nn.functional as F
from torch.nn import Module, Linear, Conv2d, MaxPool2d, BatchNorm2d, ReLU, Sequential, ConvTranspose2d, Tanh

BS = 7 # batch size
NP = 5 # number of points in the input

INS = 2
OUTS = 3

class Net(Module):
    def __init__(self):
        super().__init__()
        self.lin = Linear(INS, OUTS)
    def forward(self, x): return self.lin(x)


n = Net()

x = torch.randn(BS, NP, INS) # input tensor
x = x.view(BS * NP, INS) # reshape to (BS * NP, INS)
y = n(x) # forward pass
y = y.view(BS, NP, OUTS) # reshape back to (BS, NP, OUTS)

print(f'In  -> {x.shape}')
print(f'Out -> {y.shape}')

In [ ]:
# testing another batched input test
import torch
import torch.nn.functional as F
from torch.nn import Module, Linear, Conv2d, MaxPool2d, BatchNorm2d, ReLU, Sequential, ConvTranspose2d, Tanh

BS = 7 # batch size
NPTS = 5 # number of points in the input
PX = 4 # physical input size

LS = 4 #latent size

def concat_pts_ph(pts, ph):
    # pts: (BS, NP, 2)
    # ph: (BS, PX)
    assert pts.dim() == 3, f'Expected pts to be of shape (BS, NPTS, 2), got {pts.shape}'
    bs, npts = pts.shape[:2]
    # Reshape ph to (BS, 1, PX) and expand to (BS, NPTS, PX)
    assert ph.dim() == 2, f'Expected ph to be of shape (BS, PX), got {ph.shape}'
    x_expanded = ph.view(bs, 1, PX).expand(bs, npts, PX)
    # Concatenate along the last dimension
    out = torch.cat([pts, x_expanded], dim=-1)  # (B, H, 15)
    return out
    
x = torch.rand(BS, PX)
pts = torch.rand(BS, NPTS, 2) # input tensor of points 
# y1 = conc(pts, x) # concatenate points and physical input
y1 = concat_pts_ph(pts, x) # concatenate points and physical input


print(f'x   -> {x.shape}')
print(f'pts -> {pts.shape}')
print(f'y1  -> {y1.shape}')